In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle as pkl
import imageio
import time
import os

In [9]:
def file_name(lattice_size,J,h,T,seed):
    return f'SQ_L_{lattice_size}_J_{J:.2f}_h_{h:.2f}_T_{T}_s_{seed}'


def read_data(lattice_size, J,h,T_init,T_final,seed,num_samples):
    TEMPERATURE_SCALE = 1000
    MAIN_DIRECTORY = "ising_data"
    # We define the T_index as the index number of the values of T
    # written in the file name, seperated by "_". So in this case e.g.
    # file_name = ...._T_1545_seed_1001 
    # index: 2 gives value of lattice size
    # index: 10 gives value of "seed"
    # index from last: 3 gives 1545 i.e. the value of T
    
    L_INDEX = 2
    J_INDEX = 4
    H_INDEX = 6
    T_INDEX = 8
    SEED_INDEX = 10

    # We scale up the T values to search and parse directories
    scale_up_T_init = T_init * TEMPERATURE_SCALE
    scale_up_T_final = T_final * TEMPERATURE_SCALE
    
    # Get a numpy array of all the subdirectories
    sub_directories = np.asarray([f.path for f in os.scandir(MAIN_DIRECTORY) if f.is_dir()])
    # Now we parse the T values from the directory names
    parsed_T_values = np.zeros(sub_directories.size)
    
    for dir_names in np.arange(sub_directories.size):
        # First we split the directory names with "_"
        sub_directories[dir_names] = sub_directories[dir_names].lstrip(MAIN_DIRECTORY + "/")
        split_directory_name = np.asarray(sub_directories[dir_names].split("_"))
        # Now we take the file names that fit our needs
        condition = split_directory_name[L_INDEX].astype(int) == lattice_size and \
                    split_directory_name[J_INDEX].astype(float)==J and \
                    split_directory_name[H_INDEX].astype(float) == h and \
                    split_directory_name[SEED_INDEX].astype(int) == seed
        if condition:
            # Now we take the T values. 
            parsed_T_values[dir_names] = split_directory_name[T_INDEX]
    
    # Now we drop the values that are not in the [T_init,T_final] interval
    # First, all the values less than T_init will be changed to zero
    parsed_T_values[np.where(parsed_T_values<scale_up_T_init)]=0
    
    # Similarly, all the values greater than T_final will be changed to zero
    parsed_T_values[np.where(parsed_T_values>scale_up_T_final)]=0

    # Now we trim all the zeros, and make sure everything is sorted as integers
    # Since our file names are integeres
    parsed_T_values = np.trim_zeros(np.sort(parsed_T_values.astype(int)))
    os.chdir(MAIN_DIRECTORY)
    # Now we have everything needed to create our filenames and read it
    data = {}

    if parsed_T_values.size > num_samples:
        # Randomly choose among the T values we have
        random_indexes = np.random.choice(parsed_T_values.size, num_samples, replace=False)
        for files in np.arange(num_samples):
            random_T = parsed_T_values[random_indexes[files]]
            directory_name = file_name(lattice_size, J,h,random_T,seed)
            os.chdir(directory_name)
            key_name = str(random_T)
            open_file = open(directory_name + ".pkl",'rb')
            data_sample = pkl.load(open_file)
            data.update({key_name: data_sample})
            os.chdir('..')
    else:
        for files in np.arange(parsed_T_values.size):
            directory_name = file_name(lattice_size, J,h,parsed_T_values[files],seed)
            os.chdir(directory_name)
            key_name = str(parsed_T_values[files])
            open_file = open(directory_name + ".pkl",'rb')
            data_sample = pkl.load(open_file)
            data.update({key_name: data_sample})
            os.chdir('..')
    os.chdir("..")
    return data


def get_configurations(lattice_size, J,h,T_init,T_final,seed,num_samples):
    # First we get the data
    get_data = read_data(lattice_size, J,h,T_init,T_final,seed,num_samples)
    if get_data == []:
        print("There is no such data.")
        return None,None
    # Let us extract the keys i.e. temperatures (scaled up temperatures)
    keys = np.array(list(get_data.keys())).astype(int)
    # Initialize configurations array
    configurations = np.zeros((keys.size, get_data[f"%d"%keys[0]]['lattice_configuration'].shape[0] ,lattice_size, lattice_size))
    
    # We iterate to get all the configurations in one array  
    for i in np.arange(keys.size):
        configurations[i] = get_data[keys[i].astype(str)]['lattice_configuration']
        
    # Now first element of keys array, is the temperature of
    # the first element of configurations array.
    return keys, configurations

In [14]:
sub_directories = np.asarray([f.path for f in os.scandir("ising_data") if f.is_dir()])

In [49]:
temperatures, configurations = get_configurations(lattice_size = 128 ,
                                                  J = 1.0 ,
                                                  h = 0 ,
                                                  T_init = 1.0 ,
                                                  T_final = 3.52 ,
                                                  seed = 0 , 
                                                  num_samples = 2000)

for s in np.arange(1,9):
    temp, conf = get_configurations(lattice_size = 128 ,
                                                  J = 1.0 ,
                                                  h = 0 ,
                                                  T_init = 1.0 ,
                                                  T_final = 3.52 ,
                                                  seed = s , 
                                                  num_samples = 2000)
    configurations = np.concatenate([configurations,conf],axis=1)


In [50]:
configurations.shape

(5, 189, 128, 128)